**Ссылка на презентацию**
https://docs.google.com/presentation/d/1zOhfYL4FxUq6HUbuuWalmLLART8HLKorKEknmvfYgOs/edit?usp=sharing
https://disk.yandex.ru/i/krOSxku3QXwG0w

## Загрузка данные и подготовка их к анализу
### Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
import datetime as dt

import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go
from plotly.offline import iplot, init_notebook_mode

from io import BytesIO
import requests
import re

import warnings
warnings.simplefilter('ignore')
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [2]:
# задаем настройки отображения
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_colwidth = 150 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 200)

In [3]:
# загружаем дата-сет
try:
    rest_data = pd.read_csv('/datasets/rest_data.csv')
except:
    rest_data = pd.read_csv('https://code.s3.yandex.net/datasets/rest_data.csv')

In [4]:
# фунция для получения общей информации о датасетах
def get_info(df, df_name):
    print(f'Общая информация о таблице - {df_name}')
    display(df.tail(20)) # выводим "голову" таблицы
    print('*'*50)
    print(df.info() , '\n') # общую информацию
    print('*'*50)
    print(df.columns, '\n') # названия столбцов
    print('*'*50)
    print(df.describe(),'\n') # числовое описание
    print('*'*50)
    print(f'Количество NaN значений в таблице - {df_name}', '\n') 
    print(df.isna().sum()) # количество пропусков
    print('*'*50)
    print(f'Число дубликатов в таблице - {df_name}', '\n') 
    print(df.duplicated().sum()) # количество дубликатов

In [5]:
get_info(rest_data, 'rest_data')

Общая информация о таблице - rest_data


,id,object_name,chain,object_type,address,number
15346,208599,Мята Lounge,да,кафе,"город Москва, Куликовская улица, дом 1А",30
15347,222491,Кальянная «Мята Lounge»,да,кафе,"город Москва, Профсоюзная улица, дом 142, корпус 1, строение 1",40
15348,212216,Мята Lounge,да,кафе,"город Москва, Привольная улица, дом 11",56
15349,206341,Мята Lounge,да,кафе,"город Москва, Салтыковская улица, дом 7Г",100
15350,213061,Мята,да,кафетерий,"город Москва, Каширское шоссе, дом 96, корпус 1",35
15351,223036,Якитория,да,ресторан,"город Москва, Авиационная улица, дом 66",92
15352,213602,Тануки,да,кафе,"город Москва, Привольная улица, дом 65/32",50
15353,213772,Тануки,да,ресторан,"город Москва, Осенний бульвар, дом 9",98
15354,210400,Шоколадница,да,кафе,"город Москва, Театральный проезд, дом 5, строение 1",45
15355,74972,Шоколадница,да,кафе,"город Москва, улица Новый Арбат, дом 13",30


**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB
None 

**************************************************
Index(['id', 'object_name', 'chain', 'object_type', 'address', 'number'], dtype='object') 

**************************************************
              id    number
count  15,366.00 15,366.00
mean  119,720.07     59.55
std    73,036.13     74.74
min       838.00      0.00
25%    28,524.00     12.00
50%   144,974.50     40.00
75%   184,262.25     80.00
max   223,439.00  1,700.00 

***************

#### Вывод о загрузке данных

В таблице 6 столбцов:

* `object` - 4 столбец
* `int64` - 2 столбца

Согласно документации к данным:

* `id` — идентификатор объекта;
* `object_name` — название объекта общественного питания;
* `chain` — сетевой ресторан;
* `object_type` — тип объекта общественного питания;
* `address` — адрес;
* `number` — количество посадочных мест.

Таблица содержит 15366 строк, не имеет пропусков и явных дубликатов. Столбец 'chain' зачем-то текстовый, переведем для удобства в булевый. Столбец 'object_type' переведем в категориальный тип. Впечатляющие показатели количества посадочных мест от 0 в кофе с собой до 1700, может банкетный зал, а может быть и выброс. В названиях колонок все нормально, а в названиях ресторанов - каша, попробуем разобраться. Проверим неявные дубликаты.

В предобработке данных:

* меняем типы данных

* ищем неявные дубликаты

* проверяем посадочные места

### Предобработка данных

In [6]:
# поищем неявные дубликаты
print('Значения столбцов по возрастанию встречаемости:') # уникальные значения по возрастанию встречаемости
for value in rest_data:
    print(rest_data[value].value_counts().sort_values())

Значения столбцов по возрастанию встречаемости:
151635    1
21065     1
23670     1
24309     1
21894     1
         ..
208537    1
209264    1
209186    1
193364    1
222535    1
Name: id, Length: 15366, dtype: int64
ТБИЛИСИ                                          1
НЭЦКЭ                                            1
ЛЕ СПА                                           1
ИНТЕРНАТ №72 «КОМБИНАТ ДОШКОЛЬНОГО ПИТАНИЯ»      1
Сыто пьяно                                       1
                                              ... 
Шоколадница                                    142
KFC                                            155
Шаурма                                         234
Кафе                                           236
Столовая                                       267
Name: object_name, Length: 10393, dtype: int64
да      2968
нет    12398
Name: chain, dtype: int64
магазин (отдел кулинарии)             273
закусочная                            360
кафетерий                             

Идентификаторы у всех разные, но, кажется, четкого регламента оформления для названий нет: разные языки в названиях, много сетевых ресторанов, у которые могут отличаться оформлением.

In [7]:
# проверим строки на дубликаты без учета идентификатора
#rest_data[rest_data[['object_name','chain','object_type','address','number']].duplicated()].sort_values(by='address')
print(len(rest_data[rest_data[['object_name','chain','object_type','address','number']].duplicated()]))
print(len(rest_data))

82
15366


82 дубликата, в которых совпадает все, кроме id, вероятно, наша база данных собрана из нескольких и данные задублировались под разными идентификаторами. Удалим их.

In [8]:
rest_data = rest_data.drop_duplicates(['object_name','chain','object_type','address','number']).reset_index(drop = True)

In [9]:
# приведем столбец object_name к нижнему регистру
rest_data['object_name'] = rest_data['object_name'].str.lower()

# уберем лишние символы
rest_data['object_name'] = rest_data['object_name'].str.replace('«|»', '', regex=True)
rest_data['object_name'] = rest_data['object_name'].str.replace('-| ', '_', regex=True)

# проверим дубли по ключевому слову "мята_lounge" 
print(rest_data[rest_data['object_name'].str.contains('мята_lounge', regex=False)]['object_name'].value_counts())

мята_lounge                        24
кальянная_мята_lounge               5
кальян_бар_мята_lounge              2
кафе_мята_lounge_кальянный_клуб     1
кальян__бар_мята_lounge             1
мята_lounge_шаболовка               1
мята_lounge_автозаводская           1
лаундж_бар_мята_lounge              1
бар_мята_lounge                     1
мята_lounge_октябрьская             1
Name: object_name, dtype: int64


Как и ожидалось - шквал дублей. У сетевых ресторанов нет строгой формы записи названий, получились целые новые сети.

In [10]:
# переводим сети в булев тип
rest_data['chain'].replace({'да':True,'нет':False},inplace = True)
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15284 entries, 0 to 15283
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15284 non-null  int64 
 1   object_name  15284 non-null  object
 2   chain        15284 non-null  bool  
 3   object_type  15284 non-null  object
 4   address      15284 non-null  object
 5   number       15284 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 612.1+ KB


In [11]:
# переводим тип ресторана в категориальный тип
rest_data['object_type'] = rest_data['object_type'].astype('category')
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15284 entries, 0 to 15283
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           15284 non-null  int64   
 1   object_name  15284 non-null  object  
 2   chain        15284 non-null  bool    
 3   object_type  15284 non-null  category
 4   address      15284 non-null  object  
 5   number       15284 non-null  int64   
dtypes: bool(1), category(1), int64(2), object(2)
memory usage: 508.0+ KB


И это снизило использование памяти с 492.7 до 388.6

In [12]:
# посмотрим на сетевые рестораны поближе
rest_data.query('chain == True')['object_name'].value_counts().sort_values(ascending=False)

шоколадница                     157
kfc                             155
макдоналдс                      150
бургер_кинг                     137
теремок                          94
                               ... 
кондитерия_тирольские_пироги      1
florentini                        1
пикколо                           1
beverly_hills_diner               1
мята                              1
Name: object_name, Length: 549, dtype: int64

Сети из одного ресторана выглядят нелогично. В названиях ресторанов периодически встречается тип заведения, но мы уже имеем столбец 'object_type', избавимся от приставок в названиях.

In [13]:
# посмотрим на имена сетевых ресторанов
rest_data.query('chain == True')['object_name'].sort_values().unique()

array(['beverly_hills_diner', 'bierloga', 'black_&_white', 'bocconcino',
       'boobo', 'bubbleology', 'burger_club', 'coffeeshop',
       'coffeeshop_company', 'cofix', 'cookhouse', 'correas',
       'costa_coffee', 'deli_by_prime_прайм_кафе', 'dunkin_donuts',
       'florentini', 'fridays', 'fridays_kfc', 'glowsubs_sandwiches',
       'goodman_гудман', 'grand_cru', 'grand_урюк', 'healthy_food',
       'il_forno_иль_форно', 'jeffreys_coffee', 'kfc',
       'kfc_волгоградский', 'krispy_creme', 'krispy_krem', 'lavkalavka',
       'luciano', 'm_cafe_хинкальная', 'maki_maki', 'marmalato',
       'marrakesh_хинкальная', 'moskalyan', 'movenpick', 'my_box',
       'osteria_mario', 'panda_express', 'paul_поль', 'pizengof99_спб',
       'prime_star_прайм_стар', 'prime_прайм_стар',
       'prime_прайм_стар_прайм_кафе', 'starbucks',
       'starbucks,_старбакс_кофе', 'starbucks_coffee', 'starbucks_кофе',
       'starbucks_старбакс', 'starlite_diner', 'subway', 'sushilka',
       'tajj_mahal', '

In [ ]:
# словарь для сокращения повторов
rest_name = {'.*в&в_бургер.*' : 'в&в_бургер',
             '.*burger_club.*|бургер_кл*б' : 'burger_club',
             '.*cofix.*|.*кофикс.*':"cofix",
             '.*costa_coffee.*' : 'costa_coffee',
             '.*dunkin_donuts.*|.*д*нкин_донатс.*' : 'dunkin_donuts',
             '.*goodman.*|.*гудман.*' : 'goodman',
             '.*florentini.*' : 'florentini',
             '.*fridays.*' : 'fridays',
             '.*ikea.*' : 'ikea',
             '.*kfc.*|.*кфс*.':'kfc',
             '.*krispy_*rem*' : 'krispy_creme',
             '.*maki_maki.*|.*маки_маки.*' : 'maki_maki',
             '.*moskalyan.*' : 'moskalyan',
             '.*mcdonalds.*|.*макд.*':'mcdonalds',
             '.*starbucks.*|.*старб.*':"starbucks",
             '.*subway.*|.*сабв.й.*':"subway",
             '.*travelers_coffe.*' : 'travelers_coffee',
             '.*tutti_frutti.*' : 'tutti_frutti',
             '.*upside_down.*' : 'upside_down',
             '.*van_wok.*|.*ван*вок.*' : 'van_wok',
             '.*vietcafe.*|.*вьеткафе.*' : 'vietcafe',
             '.*азбука_вкуса.*':"азбука_вкуса",
             '.*академия.*' : 'пиццерия_академия',
             '.*алло_пицца.*':"алло_пицца",
             '.*андерсон.*':"андерсон",
             '.*американ_сити.*' : 'американ_сити_пицца',
             '.*бар.*буфет.*николай*.' : 'николай',
             '.*бакинский_бульвар.*' : 'бакинский_бульвар',
             '.*баскин_роббинс.*':"баскин_роббинс",
             '.*бенто_wok.*' : 'бенто_wok',
             '.*бир_хаус.*' : 'бир_хаус',
             '.*бургер_кинг.*|.*burger_king.*':"burger_king",
             '.*брусника.*':"брусника",             
             '.*ваби_саби.*':"ваби_саби",
             '.*вареничная.*1.*':"вареничная_№1",
             '.*волконский.*':"волконский",
             '.*wokker.*|.*воккер.*' : 'каффе_wokker',
             '.*грабли.*':"грабли",
             'грузинские_каникулы_барбарис' : 'барбарис',
             '.*дабл_би.*' : 'дабл_би',
             '.*да_пино.*|.*da_pino*.' : 'да_пино',
             '.*де_марко*.' : 'де_марко',
             '.*джон_джоли.*':"джон_джоли",
             '.*додо.*':"додо_пицца",
             '.*домино.*':"домино'с_пицца",
             '.*ёрш.*' : 'ресторан_ёрш',
             '.*зю_кафе.*' : 'зю_кафе',
             '.*иль_патио.*':"иль_патио",
             '.*иль_форно.*|.*il_forno.*' : 'il_forno',
             '.*космик.*' : 'космик',
             '.*кофемания.*':"кофемания",
             '.*кофе.*тун.*' : 'кофе_тун',
             '.*кофе_хаус.*':'кофе_хаус',
             '.*кофешоп.*|.*coffeeshop.*':"coffeeshop_company",
             '.*крошка_картошка.*':'крошка_картошка',
             '.*кружка.*':"кружка",
             '.*брать.*караваев.*':"кулинарная_лавка_братьев_караваевых",
             '.*кулинарное_бюро.*' : 'кулинарное_бюро_kitchen',
             '.*лукойл.*' : 'лукойл',
             '.*магбургер.*' : 'магбургер',
             '.*меленка.*' : 'меленка',
             '.*милти.*':"милти",
             '.*мимино.*' : 'мимино',
             '.*мск_московская_сеть_кальянных.*' : 'мск_московская_сеть_кальянных',
             '.*му_му.*':"му_му",
             '.*мята.*':"мята_lounge",
             '.*нияма.*' : 'нияма_пицца_пи',
             '.*ньокки.*' : 'ньокки',
             '.*папа_джонс.*':"папа_джонс",
             '.*пилзнер.*' : 'пилзнер',
             '.*пицца_паоло.*':"пицца_паоло",
             '.*пицца_pomodoro.*' : 'пицца_pomodoro',
             '.*пицца_хат.*|.*pizza_hut.*':"pizza_hut",
             '.*планета_суши.*' : 'планета_суши',
             '.*прайм.*|.*prime.*star.*':"прайм_стар",
             '.*пронто.*' : 'ресторан_пронто',
             '.*ста.*дог.*':"стардогs",
             '.*советские_времена.*' : 'советские_времена',
             '.*старина_миллер.*' : 'старина_миллер',
             '.*суши_вок.*|.*shushi_wok.*|.*суши_wok.*|.*shushi_wok.*':"суши_wok",
             '.*суши_сет.*|.*суши_set.*|.*shushi_set.*':"суши_сет",
             '.*суши_тун.*' : 'суши_тун',
             '.*суш.*шоп.*|.*shushishop.*':"суши_шоп",
             '.*тарас_бульба.*':"тарас_бульба",
             '.*тануки.*':"тануки",
             '.*тапчан.*' : 'кафе_чайхана_тапчан',
             '.*теремок.*' : 'теремок',
             'ресторан_территория.*|территория_яс.*|.*территория_tim.*|территория' : 'ресторан_территория',
             '.*ян_примус' : 'ресторан_ян_примус',
             '.*тирольские_пироги.*' : 'тирольские_пироги',
             '.*то.да.с.*':"то_да_сё",
             '.*торро_гриль.*' : 'торро_гриль',
             '.*урюк.*' : 'урюк',
             '.*шантимель.*' : 'шантимель',
             '.*шоколадница.*':'шоколадница',
             '.*штолле.*':"штолле",
             '.*чайхона.*1.*':"чайхона_№1",
             '.*хлеб_насущный.*':"хлеб_насущный",
             '.*якитория.*':"якитория"}


for name, new_name in rest_name.items():
    rest_data['object_name'] = rest_data['object_name'].str.replace(name, new_name, regex='last')

In [ ]:
# проверяем работу замены
rest_data[rest_data['object_name'].str.contains('мимино')]

Так имена меняют некоторые несетевые организации, но пополняются сети

In [ ]:
# посмотрим как изменились сети
rest_data.query('chain == True')['object_name'].sort_values(ascending=False).value_counts().head()

Основные сети "подросли". Пятерка лидеров особо не изменилась, разве что, кфс стало больше, чем шоколадницы. Все еще есть сети, в которых один ресторан.

In [ ]:
# оценка сетей, где 1 ресторан 
rest_data[rest_data['chain'] == True]['object_name'].value_counts().sort_values(ascending=True).head(104)

Вряд ли могут быть сетевые рестораны с 1 заведением, возможно это первые филиалы и франчайзи из успешных сетей в других регионах. Исправим это.

In [ ]:
# соберем список сетевых ресторанов с 1 заведением
rest_one = list(rest_data[rest_data['chain'] == True]['object_name'].value_counts().sort_values(ascending=True).head(103).index)

In [ ]:
# достанем индексы этих ресторанов
rest_one_position = list(rest_data.loc[rest_data['object_name'].isin(rest_one)]['chain'].index)

In [ ]:
# функция для замены булевых значений  
def reset_chain(rest_one_position):
    rest_data.loc[rest_one_position,'chain'] = False
reset_chain(rest_one_position)

In [ ]:
# проверяем работу замены
rest_data[rest_data['object_name'].str.contains('гино_но_таки')]

Сделали сетевые рестораны несетевыми.

In [ ]:
# ищем дубли по адресам, количеству посадочных мест и типу ресторана
print('Количество дублей по адресу:', rest_data[['object_name', 'address']].duplicated().sum())
print('Количество дублей по адресу и количеству посадочных мест:', 
      rest_data[['object_name', 'address', 'number']].duplicated().sum())
print('Количество дублей по адресу, типу и количеству посадочных мест:', 
      rest_data[['object_name', 'address', 'number','object_type']].duplicated().sum())

In [ ]:
# посмотрим на повторы
rest_data[rest_data.duplicated(subset=['object_name','address','number','object_type'], keep=False)]

После удаления 82 дубликатов по поиску без идентификатора, даже без приведения адреса к нижнему регистру осталось три полных дубля.

In [ ]:
# убираем тройку полных совпадений
rest_data = rest_data.drop_duplicates(subset=['object_name','address','number','object_type'])

In [ ]:
# посмотрим кто остался похожим, но без типа организации 
rest_data[rest_data.duplicated(subset=['object_name','address','number'], keep=False)]

Выглядят довольно одинаково.

In [ ]:
# удалим их тоже
rest_data = rest_data.drop_duplicates(subset=['object_name','address','number'])
print('Количество совпадений точек одной сети по адресу и количеству посадочных мест:', 
      rest_data[['object_name', 'address', 'number']].duplicated().sum())

Только по адресу смотреть не будем, поскольку несколько ресторанов, даже одной сети, могут быть в одном здании.

In [ ]:
# строим диаграмму размаха
sns.set(rc={'figure.figsize':(9,9)})
sns.set_theme()
sns.boxplot(y="number", data=rest_data)
plt.title('Диаграмма размаха количества посадочных мест в заведении', fontsize=14)
plt.ylabel('Количество посадочных мест', fontsize=12)

Из числового описания и диаграммы размаха видим: в среднем мест 60, в медиане 40. 

In [ ]:
# рассчитаем верхнюю границу выбросов
q1 = rest_data['number'].quantile(0.25)
q3 = rest_data['number'].quantile(0.75)
iqr = q3 - q1 # iqr

q3 + 1.5 * iqr

Сделаем срез чуть выше верхней границы выбросов и посмотрим распределение основной массы ресторанов.

In [ ]:
# строим гистограмму
rest_data['number'].hist(figsize=(9,9), density = True, bins=150)
rest_data['number'].plot(kind = "kde", linewidth=3, color='salmon')
plt.xlabel('Количество посадочных мест')
plt.ylabel('Частота встречаемости')
plt.title('Распределение количества посадочных мест')
plt.xlim(0, 200)
plt.grid(True)
plt.show()

Чаще всего в заведениях от нуля до двадцати мест, 75% объектов имеют не больше 80 мест.

In [ ]:
# посмотрим кто стоит за большими залами
rest_data[['object_name', 'object_type','address', 'number']].query('number > 800').sort_values(by='number', ascending=False).style.background_gradient(cmap='magma') 

8 самых больших залов. Банкетные залы и рестораны похожи на правду. Бар с названием arena_by_soho_family может оказаться чем-то вроде концертного зала с баром. Столовые выглядят впечатляюще, являются ли они заводскими столовыми или выбросами?

#### Вывод о предобработке данных

* изменили типы данных в столбце 'chain' с текстового на булевый, и в столбце 'object_type' с текстового на категориальный 

* почистили таблицу от неявных дубликатов, изменили сетевые рестораны с одним заведением в сети на несетевые

* проверили посадочные места на адекватность

## Анализ данных

### Соотношение видов объектов общественного питания по количеству

In [ ]:
# собираем таблицу c расчетом видов объектов общественного питания по количеству
rest_type = rest_data.pivot_table(
    index='object_type',
    values='id',
    aggfunc='count'
).sort_values(by='id',ascending=False).reset_index()

In [ ]:
# строим столбчатую диаграмму
plt.figure(figsize=(12,9))


ax = sns.barplot(
    x='id', 
    y='object_type', 
    data=rest_type,
    color="#4169E1",
    order=[
        "кафе", 
        "столовая",
        "ресторан", 
        "предприятие быстрого обслуживания",
        "бар",
        "буфет",
        "кафетерий",
        "закусочная",
        "магазин (отдел кулинарии)"
    ])

ax.set_title('Соотношение видов объектов общественного питания по количеству', fontsize = 14)
ax.set_xlabel('Количество объектов') 
ax.set_ylabel('Тип объекта')

for p in ax.patches:
                _x = p.get_x() + p.get_width() + float()
                _y = p.get_y() + p.get_height() - float()
                value = int(p.get_width())
                ax.text(_x, _y, value, ha="left")

sns.despine()


Больше всего кафе - 6067, далее столовые - 2584 и рестораны - 2281
Меньше всех магазины(отдел кулинарии) - 273

### Соотношение сетевых и несетевых заведений по количеству

In [ ]:
# собираем таблицу c расчетом сетевых и несетевых заведений по количеству
chain = rest_data.pivot_table(index='chain',values='id',aggfunc='count')
chain = chain.reset_index()
chain.columns=['chain','count']

# заменим цифровые обозначения на текст для лучшего восприятия графика
chain['chain'] = chain['chain'].replace(True,'Сетевое')
chain['chain'] = chain['chain'].replace(False,'Несетевое')

In [ ]:
df = px.data.tips()
fig = px.pie(chain, 
             values='count', 
             names='chain',
             labels='chain',
             color='chain', 
             color_discrete_map={
                "Сетевое": "ff977e",
                "Несетевое": "721f81"},
            title='Соотношение сетевых и несетевых заведений',
            hover_name='chain',
            height=350)
fig.show()

* 81.3% (12413 заведений) - несетевые
* 18.7% (2859 заведений) - сетевые

На рынке преобладают несетевые организации, их в 4 с небольшим раза больше, чем несетевых.

### Характер сетевого распространения по виду объекта общественного питания

In [ ]:
# собираем таблицу
rest_pivot = rest_data.pivot_table(index='object_type',
                                   columns='chain',
                                   values='id',
                                   aggfunc='count'
                                  ).sort_values(by=False,ascending=False).reset_index()

rest_pivot.columns=['Тип_Заведения','Несетевой','Сетевой']
rest_pivot['Доля сетевых'] = round((rest_pivot['Сетевой'] / (rest_pivot['Сетевой']+rest_pivot['Несетевой']) * 100), 2)
rest_pivot['Доля несетевых'] = round((rest_pivot['Несетевой'] / (rest_pivot['Сетевой']+rest_pivot['Несетевой']) * 100), 2)
rest_pivot

In [ ]:
# строим гистограмму
plt.set_title='213'
fig = go.Figure(data=[
    go.Bar(name='Сетевой', x=rest_pivot['Тип_Заведения'], y=rest_pivot['Сетевой'], marker_color='#ff977e'),
    go.Bar(name='Несетевой', x=rest_pivot['Тип_Заведения'], y=rest_pivot['Несетевой'], marker_color='#721f81')
])

# задаем тип гистограммы
fig.update_layout(barmode='group')


In [ ]:
# строим гистограмму

fig = go.Figure(data=[
    go.Bar(name='Сетевой', x=rest_pivot['Тип_Заведения'], y=rest_pivot['Доля сетевых'], marker_color='#ff977e'),
    go.Bar(name='Несетевой', x=rest_pivot['Тип_Заведения'], y=rest_pivot['Доля несетевых'], marker_color='#721f81')
])
 
# задаем тип гистограммы
fig.update_layout(barmode='group')

Из таблицы и графика видим, что: 
* самый высокий процент сетевых ресторанов обосновалисть в предприятиях быстрого обслуживания - 41.17% 
* далее - магазины (отделы кулинарии) - 27.11%, 
* третье место практически наравне делят рестораны - 22.31% и кафе - 22.17%

Самые низкие проценты сетевых заведений среди столовых и буфетов. Стоит отметить, что часто такие организации бывают закрытого типа, вроде "столовая при заводе" или "буфет университета", куда "людей с улицы" не пускают.

### Характер сетевых заведений по числу посадочных мест и количеству заведений

In [ ]:
# соберем таблицу
chain_number = rest_data.query("chain == True").groupby('object_name').agg({'id': 'count', 'number':'median'})
chain_number.head()

In [ ]:
# найдем границу "много"
print(np.percentile(chain_number['id'], [75,95,99])) 
print(np.percentile(chain_number['number'], [75,95,99]))

* не более 25% заведений имеют 11 точек в сети и 82 посадочных места
* 5% заведений имеют 69 точек и 141 посадочное место
* 1% заведений имеют 173 точки и 207 посадочных мест

Мы ищем "типичных" представителей, поэтому за "много" возьмем 75 процентиль.

In [ ]:
# создаем переменные для большого количества точек и мест
big_chain = np.percentile(chain_number['id'], [75])
big_number = np.percentile(chain_number['number'], [75])

# разбиваем на группы количество точек в сети
chain_number['rest_number'] = 'Мало точек'
chain_number.loc[chain_number['id'] > big_chain[0], 'rest_number'] = 'Много точек'

# разбиваем на группы количество посадочных мест
chain_number['seat_number'] = 'Мало посадочных мест'
chain_number.loc[chain_number['number'] > big_number[0], 'seat_number'] = 'Много посадочных мест'

# сделаем столбец group
chain_number['group'] = chain_number['rest_number'].astype('str') + ' - ' + chain_number['seat_number'].astype('str')

In [ ]:
# перегруппируем таблицу
chain_number.reset_index().groupby('group').agg({'object_name':'nunique', 'id': 'median', 'number':'median'})\
.rename(columns = {'object_name' : 'Количество заведений в сети',
                  'id' : 'Медианное число точек',
                  'number': 'Медианное число посадочных мест'}).reset_index()


In [ ]:
# строим график
fig = px.scatter(chain_number, x="id", y="number", color="group",
             color_discrete_sequence=['#3b0f70', '#8c2981', '#de4968', '#fe9f6d'],
             title="Распределение количества точек и количества посадочных мест в сетевых заведениях"
            )

fig.show()

Видим, что для сетевых заведений наиболее характерно мало точек и мало посадочных мест, таких сетей у нас 104. У нас определенно много маленьких сетей в дата-сете.

In [ ]:
#построим график совместного распределения
plt.figure(figsize=(14,9))

ax = sns.jointplot(data=chain_number, 
              x='id', 
              y='number', 
              kind='reg', 
              color='#8c2981')

plt.suptitle("Взаимосвязь числа посадочных мест в заведении и размера сети", fontsize = 14)
plt.subplots_adjust(top=0.9)
ax.ax_joint.set_xlabel('Количество точек в сети')
ax.ax_joint.set_ylabel('Количество посадочных мест')
sns.despine()

In [ ]:
chain_number[['id', 'number']].reset_index().sort_values('id',ascending=False).head()

График показывает обратную зависимость между числом заведений в сети и количеством посадочных мест в каждом объекте: чем меньше в сети заведений, тем они вместительнее, тогда как крупные сети, такие как kfc, шоколадница, burger_king или теремок имеют в медиане до 50 посадочных мест.

###  Среднее количество посадочных мест для каждого вида объекта общественного питания

In [ ]:
# собираем таблицу, считаем среднее
number_mean = rest_data.pivot_table(index='object_type', values='number',aggfunc='median').reset_index()
number_mean.columns=['object_type','median']
number_mean = number_mean.sort_values(by='median',ascending=False)

In [ ]:
# рисуем гистограмму
ax = sns.barplot(
    x='median', 
    y='object_type', 
    data=number_mean,
    color="#4169E1",
    order=[
        "столовая", 
        "ресторан",
        "буфет", 
        "бар",
        "кафе",
        "предприятие быстрого обслуживания",
        "кафетерий",
        "закусочная",
        "магазин (отдел кулинарии)"
    ])

ax.set_title('Соотношение видов объектов общественного питания по количеству посадочных мест', fontsize = 14)
ax.set_xlabel('Количество посадочных мест') 
ax.set_ylabel('Тип объекта')

for p in ax.patches:
                _x = p.get_x() + p.get_width() + float()
                _y = p.get_y() + p.get_height() - float()
                value = int(p.get_width())
                ax.text(_x, _y, value, ha="left")

sns.despine()


Мы не очищали таблицу от выбросных значений по местам, поэтому для расчета среднего использовали устойчивую к выбросам медиану.

* 103 мест в среднем - самыми большими по количеству мест являются столовые
* 80 мест в среднем - второе место у ресторанов
* 35-30 мест в среднем - буфеты, бары и кафе

Магазины (отделы кулинарии) и закусочные в среднем не имеют сидячих мест.

In [ ]:
# выделим название улиц отдельно
rest_data['street'] = rest_data['address'].apply(lambda x: x.split(',')[1]) 
rest_data['street'] = rest_data['street'].str.strip()
rest_data.sample(10)

Обрезали строку с адресом по первой запятой и получили улицу "город Зеленоград", посмотрим его поближе

In [ ]:
# посмотрим на запись улиц в Зеленограде.
rest_data.query('street=="город Зеленоград"')

232 заведения в Зеленограде. Часть точек сетевые, а часть не имеет улицы как таковой: "город Москва, город Зеленоград, корпус 438"

### Строим график топ-10 улиц по количеству объектов общественного питания. Воспользуемся внешней информацией, чтобы посмотреть локацию улиц по районам

In [ ]:
# топ-10 улиц
rest_data['street'].value_counts().sort_values(ascending = False).head(10)

А вот и Зеленоград, почетное первое место, он, конечно, является округом Москвы, но сложно считать его улицей. Еще одна позиция а топе - поселение Сосенское. По отдельности улицы из этих двух мест вряд ли бы попали в топ. Топ получился не совсем корректный, так как из-за своей специфики записи адреса в таблице малые города и поселения объединяются в улицу, поскольку в записи их наименование стоит на 2 месте.

In [ ]:
# избавимся от малых городов в улицах
rest_street = rest_data.query('street not in("поселение Сосенское","город Зеленоград")').pivot_table(index='street',
                                    values='id',
                                    aggfunc='count').sort_values(by='id',ascending=False).reset_index()
top_streets = rest_street.head(10)
top_streets

In [ ]:
# посмотрим топ на диаграмме
plt.figure(figsize=(12,6))
ax = sns.barplot(x='id', y='street', data=top_streets,color="#4169E1", orient='h')
ax.set_title('Топ-10 улиц по количеству заведений')
ax.set_xlabel('Количество точек', fontsize = 12) 
ax.set_ylabel('') 
sns.despine()
plt.show()

Лидер по количеству заведений - проспект Мира. Стоит отметить, что разрыв между пунктами максимально составил 22 ресторана на улицу. 

In [ ]:
# добавим данные районов
spreadsheet_id = '1xBnX9tZ0ohnonz8Pz1Fo_pi714lENfsQEkr5M-klZtE' # указываем идентификатор таблицы
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
district = pd.read_csv(BytesIO(r.content))
district

In [ ]:
# переименуем для удобства склейки
district = district.rename(columns={'streetname':'street'})

In [ ]:
# приклеили районы и округа
district_merged = top_streets.merge(
    district[['okrug','street','area']], 
    on=['street'], 
    how='left')
district_merged.head()

In [ ]:
# группируем районы и округа по улице
district_merged.groupby('street').agg(
    okrug = ('okrug', 'unique'), 
    area = ('area', 'unique')).reset_index()

In [ ]:
# посмотрим на графике в каких округах оказались самые заполненные общепитом улицы
many_rest_area = district_merged.groupby('okrug').agg({'street':'nunique'}).reset_index()
plt.figure(figsize=(10, 5))
ax = sns.barplot(x='street', y='okrug', data=many_rest_area, color="#4169E1")
ax.set_title('Расположение улиц из Топ-10')
ax.set_xlabel('округ Москвы', fontsize = 12) 
ax.set_ylabel('количество улиц из Топ-10', fontsize = 12)
ax.set_xticks([1, 2, 3, 4])
sns.despine()
plt.show()



In [ ]:
many_rest = district_merged.groupby('area').agg({'id':'sum'}).sort_values(by = 'id', ascending = False).head(10)
many_rest


В Топ попали, вероятно, самые длинные улицы Москвы, проходящие сразу через несколько районов, а порой и округов. Логично, что в сумме на этих улицах больше всего ресторанов.

Больше всего улиц из Топ-10 в ЮАО и ЮЗАО, по 4 улицы на округ. По 3 улицы попали в ЗАО и ЦАО. И меньше всех, по одной, попали в САО и СВАО.

Самыми населенными общепитом районы вышли Район Теплый Стан, Обручевский район, Донской район.

### Считаем число улиц с одним объектом общественного питания, смотрим районы локации улиц

In [ ]:
# собираем таблицу из улиц с одним объектом общественного питания, смотрим по rest_data, чтобы вошли малые города
one_rest_street = rest_data.pivot_table(index='street',
                                    values='id',
                                    aggfunc='count').sort_values(by='id',ascending=False).reset_index()
one_rest_street = one_rest_street.query('id < 2')
one_rest_street.info(5)

In [ ]:
# приклеиваем районы и окрыга
district_merged2 = one_rest_street.merge(district[['okrug','street','area']], on=['street'], how='left')
district_merged2.isna().sum() # ищем улицы без района

In [ ]:
district_merged2 = district_merged2.dropna().reset_index() # удаляем их

In [ ]:
# посмотрим на графике в каких округах оказались самые пустые на общепит улицы
one_rest_district = district_merged2.groupby('okrug').agg(
    {'street':'nunique'}
).reset_index().sort_values('okrug', ascending=False)
plt.figure(figsize=(12, 7))
ax = sns.barplot(x='street', y='okrug', data=one_rest_district, color="#4169E1")
ax.set_title('Расположение улиц с одним объектом общественного питания', fontsize = 14)
ax.set_xlabel('округ Москвы', fontsize = 14) 
ax.set_ylabel('количество улиц с одним объектом', fontsize = 14)
sns.despine()
plt.show()
one_rest_district

In [ ]:
# выделим топ-10 районов с одним ресотраном на улице
one_rest_area = district_merged2.groupby('street',as_index=False).agg({'area':'first'})[['street','area']]
areas2 = one_rest_area.groupby('area',as_index=False).agg({'street':'count'})


areas2 = areas2.sort_values(by='street',ascending=False).head(10)
print('ТОП 10 районов с одинокими улицами:')
areas2

In [ ]:

plt.figure(figsize=(10, 5))
ax = sns.barplot(x='street', y='area', data=areas2, color="#4169E1")
ax.set_title('Расположение улиц с одним объектом общественного питания')
ax.set_xlabel('район Москвы', fontsize = 12) 
ax.set_ylabel('количество улиц с одним объектом общественного питания', fontsize = 12)
sns.despine()
plt.show()
one_rest_district

Всего получилось 579 улиц с одним заведением, 62 улицы не нашли свой район.

Лидирует по количеству улиц с одним заведением ЦАО. Вероятно, историческая застройка Москвы дает о себе знать: в центральном районе очень много маленьких улиц с ограниченным количеством домов. Больше всего таких улиц собрались в Таганском, Басманном и районе Хамовники. Меньше всего в Мещанском. А Зеленоград удивляет, всего одна улица с одним заведением. 

### Сделайте общий вывод и дайте рекомендации о виде заведения, количестве посадочных мест, а также районе расположения. Прокомментируйте возможность развития сети.

In [ ]:
# посмотрим на распределение количества посадочных мест для улиц с большим количеством объектов общественного питания
top10_rest = rest_data[rest_data['street'].isin(district_merged['street'].unique())] # выделяем топ-улицы

# строим гистограмму
fig1, fig2 = plt.subplots(figsize=(12,5))
fig1 = sns.distplot(top10_rest['number'],label = 'Топ-10 улиц', hist_kws=dict(alpha=1))
fig2 = sns.distplot(rest_data['number'],label = 'Все улицы', hist_kws=dict(alpha=0.5), color='salmon' )
fig1, fig2 = plt.xlim(-10, 300)

plt.title("Вместимость всех и ТОП улиц")
plt.xlabel('Количество посадочных мест')
plt.ylabel('Частота встречаемости')
plt.legend()
plt.show()

Графики практически идентичны. Пока можно отметить, что на топ-10 улицых чаще встречаются малые заведения до 25 мест, а на остальных улицах, чаще можно встретить точки от 100 до 125 мест и более 150 мест.

In [ ]:
top10_rest['street'].to_list()

In [ ]:
# строим бокс-плот по распределению посадочных мест на топ-10 улицах

plt.figure(figsize=(12, 7))
plt.xlim(0,220)
plt.title('Распределение количества посадочных мест (Топ-10 улиц)')
#sns.scatterplot(x="street", y="number", data=top10_rest) 
sns.boxplot(
    x="number", 
    y="street", 
    data=top10_rest, 
    color="#4169E1", 
    orient='h') 
plt.xlabel('Количество мест')
plt.ylabel('') 
plt.show()

Медианное значение по всем топ-10 улицам не превышает 50 мест, а "гиганты" (зона выбросов), в половине случаев имеют чуть больше 150 мест. Небольшое количество посадочных мест, вероятно, из-за дороговизны аренды. 

### Вывод

При планировании открытия нового бизнеса следует исходить в первую очередь из характера будущего заведения и его целевой аудитории. В условиях мы имеем небольшой, дорогой и оригинальный проект. Инвесторов, в свою очередь, интересует текущее положение дел на рынке — сможете ли вы снискать популярность на долгое время, когда все зеваки насмотрятся на роботов-официантов?

Исходя из условий в городе Москва:

* самый популярный тип заведения - кафе. Число кафе составило  6067 заведений, из них 22.17% сетевые 

* для создания сетевого бизнеса, с точки зрения данных, лучше подходит предприятие быстрого обслуживания - 41.17%, но, зачастую там не требуются официанты

* медианное число посадочных мест 40, а так же, чем больше сеть, тем меньше посадочных мест в одной точке

* самые "наполненные" общепитом улицы: на проспект Мира, Профсоюзная улица, Ленинградский проспект, Пресненская набережная, Варшавское шоссе, Ленинский проспекте, проспект Вернадского, Кутузовский проспект, Каширское шоссе и Кировоградская улица. Это протяжённые улицы: девять из десяти улиц из списка проходят через несколько муниципальных округов, при этом проспект Мира, проспект Вернадского и Варшавское шоссе пересекают по два административных округа, а Ленинский проспект проходит через четыре административных округа. Исключение составляет Пресненская набережная: высокая концентрация заведений здесь связана с ММДЦ "Москва-Сити", где располагается большое количество разнообразных объектов общественного питания. Большая часть объектов на изучаемых улицах располагается на юге, западе или в центре столицы. 

* топ-3 районов по числу заведений оказались Район Теплый Стан, Обручевский район, Донской район


Рекомендации к открытию:

* тип заведения лучше выбрать кафе. С новым форматом логичнее будет зайти не в нишевый сегмент, а в самый популярный 

* не стоит на старте вкладываться в создание сети - это поможет снизить финансовые риски. Только 22.17% кафе - сетевые, а благодаря роботам проект и так дорог

* оптимальным выглядит открытие небольшого заведения на 20-30 мест, это характерно Москвы по количеству и не придется покупать толпу "официантов"

* пресненскую набережную стоит выделить отдельно: она в топе улиц по наполненности заведениями, имеет самый низкий размах количества посадочных мест с медианой выше 25 посадочных мест. Лучше начать искать место здесь, а потом переходить на "наполненные" улицы в топ-3 районов

